<a href="https://colab.research.google.com/github/JulianaFanni/mvp-MachineLearning-Analytics/blob/main/Juliana_Fanni_MVP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVP- Juliana Fanni

Disciplina: Sprint: Machine Learning & Analytics

Profs. Hugo Villamizar e Patrick Happ

Aluna: Juliana Cardozo Pereira Fanni

## 1. Definição do Problema